# MARL EVAL Tools quickstart notebook
This notebook provides an easy introduction to the Eval repo by showing an example of how to use different plotting tools

### 1. Installing packages
We start by installing the necessary packages

In [2]:
#@title Install required packages. (Run Cell)
#git clone
#install requirement (this needs to have the pip install)


### 2. Import the necessary modules
We first import the necessary modules that will be used to processing MARL experiment data and showing the plots

In [2]:
#@title Import required packages. (Run Cell)

# In case marl_tools not detected
import sys
sys.path.insert(1, "../")


import json

from marl_tools.plotting_tools.plotting import (
    aggregate_scores,
    performance_profiles,
    probability_of_improvement,
    sample_efficiency_curves,
)
from marl_tools.utils.data_processing_utils import (
    create_matrices_for_rliable,
    data_process_pipeline,
)

### 3. Data processing and preparation

Second, we import the json file, run the processing pipeline that will help to calculate the normalized values.

In case, you don't have data and you want to test the tools, you can run generate_dummy_smac_data to generate some data

In [5]:
# Import the data
with open("./smac_dummy_data.json", "r") as data_file:
    raw_data = json.load(data_file)

# Choose the list of metrics to normalize
METRICS_TO_NORMALIZE = ["return"]

# Call data_process_pipeline to normalize the choosen metrics and to clean the data
processed_data = data_process_pipeline(
    raw_data=raw_data, metrics_to_normalize=METRICS_TO_NORMALIZE
)

# Save the processed data in different json file
with open("./smac_processed_dummy_data.json", "w+") as f:
    json.dump(processed_data, f, indent=4)

We need two different matrices to be used to create the plots:
  
  **a)  metric_dictionary_return:**

  - The output will be a dictionary that have as keys the different metrics and as a value a full other dictionary that have the different algorithm with their related metric_matrix.

  - The metric_matrix of dimenetion (number of runs x number of tasks), include the normalised absolute results.

  **b)  final_metric_tensor_dictionary:**

  - The output format will be similar to the first one except the matrix that correspond to the algorithm.

  - The matrix of dimention (number of runs x number of tasks x number of logging steps), include the normalised metrics results.

create_matrices_for_rliable function have 3 args:

  - data_dictionary: Dictionary of processed data
  
  - environment_name: Name of environment for which arrays should be computed.

  - metrics_to_normalize: List of metric names of metrics that should be normalised.

In [ ]:
environment_comparison_matrix, sample_effeciency_matrix = create_matrices_for_rliable(
    data_dictionary=processed_data,
    environment_name="SMAC",
    metrics_to_normalize=METRICS_TO_NORMALIZE,
)